In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/epilepsy/Epileptic_Seizure_Detection_in_EEG_Signals_Using_Machine_Learning_and_Deep_Learning_Techniques.pdf
/kaggle/input/research/s00259-020-05108-y.pdf


In [2]:
from huggingface_hub import login
login()

In [3]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 55.0 MB/s eta 0:00:00:00:0100:01


In [4]:
!pip install langchain langchain-community langchain-core pypdf sentence-transformers transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 868.7 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
!pip install PyPDF2


In [8]:
!pip install transformers accelerate sentencepiece PyPDF2 requests


In [10]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
# from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import requests
import io
import PyPDF2
from itertools import combinations

In [11]:
model_name = "mistralai/Mistral-Nemo-Instruct-2407"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
).eval()

tokenizer_config.json:   0%|          | 0.00/181k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.26M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/622 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/29.9k [00:00<?, ?B/s]

...

model-00005-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [12]:
def generate_text(prompt, max_length=300):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_length,
            pad_token_id=tokenizer.eos_token_id
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [13]:
pip install fitz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 42.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.4/95.4 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 425.9/425.9 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.1/565.1 kB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 88.0 MB/s eta 0:00:00:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
# import torch
# torch.cuda.empty_cache()

In [14]:
def extract_abstract(text):
    
    markers = [
        ("Abstract", "Introduction"),
        ("ABSTRACT", "1. Introduction"),
        ("Summary", "Keywords"),
        ("\nAbstract\n", "\n1 "),
        ("abstract\n", "\n1 "),
    ]
    
    text_lower = text.lower()
    for start_marker, end_marker in markers:
        start_marker_lower = start_marker.lower()
        end_marker_lower = end_marker.lower()
        
        if start_marker_lower in text_lower and end_marker_lower in text_lower:
            start_idx = text_lower.index(start_marker_lower) + len(start_marker)
            end_idx = text_lower.index(end_marker_lower, start_idx)
            return text[start_idx:end_idx].strip()
    
    return text[:1000]

In [15]:
def extract_topics(text):
    abstract = extract_abstract(text)[:1500]  
    
    prompt = f"""<s>[INST] Extract the 5 most specific research topics from this abstract.
    - Return ONLY the topics as bullet points (- topic)
    - Each topic should be 2-5 words maximum
    - Exclude generic terms like 'analysis' or 'study'
    - Do NOT include any examples or explanations
    
    ABSTRACT: {abstract} [/INST]"""
    
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    outputs = model.generate(
        **inputs,
        max_new_tokens=150,
        temperature=0.3,  
        top_p=0.9,
        do_sample=True,
        no_repeat_ngram_size=2,
        early_stopping=True
    )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
   
    topics = []
    for line in response.split('\n'):
        line = line.strip()
        if line.startswith('-'):
            topic = line[1:].strip()
            if 2 <= len(topic.split()) <= 5:  
                topics.append(topic)
    
    return topics[:5]  

In [16]:
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        for page in reader.pages:
            text += page.extract_text() or ""
    return text

# 5. التنفيذ
pdf_path = "/kaggle/input/research/s00259-020-05108-y.pdf"
paper_text = extract_text_from_pdf(pdf_path)
print("First 300 chars of extracted text:", paper_text[:300], "...")  # للتحقق

topics = extract_topics(paper_text)
print("\nExtracted Topics:")
for i, topic in enumerate(topics, 1):
    print(f"{i}. {topic}")

The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


First 300 chars of extracted text: ORIGINAL ARTICLE
A deep learning framework for18F-FDG PET imaging diagnosis
in pediatric patients with temporal lobe epilepsy
Qinming Zhang1,2&Yi Liao1,3&Xiawan Wang1,3&Teng Zhang1,3&Jianhua Feng4&Jianing Deng2&Kexin Shi1,3&
Lin Chen1&Liu Feng1,3&Mindi Ma1,3&Le Xue1,3&Haifeng Hou1&Xiaofeng Dou1&Cong ...

Extracted Topics:
1. Symmetricity-Driven Deep Learning Framework
2. Pediatric Patients with Epipsey
3. Tempora Loble Epipsy (Pediatric)
4. Symmetry Features (366)
5. Pair-of-Cube Based


In [22]:
import requests
import xml.etree.ElementTree as ET

#1) search arXiv 
def search_arxiv_by_topic(topics, max_results=3):
    base_url = "http://export.arxiv.org/api/query"
    ns = {"atom": "http://www.w3.org/2005/Atom"}
    all_results = {}

    for topic in topics:
        params = {
            "search_query": topic,
            "start": 0,
            "max_results": max_results
        }
        r = requests.get(base_url, params=params, timeout=15)
        if r.status_code != 200:
            all_results[topic] = [{"title": "Error fetching results", "link": "", "abstract": ""}]
            continue

        root = ET.fromstring(r.content)
        results = []
        for entry in root.findall("atom:entry", ns):
            title_el = entry.find("atom:title", ns)
            id_el    = entry.find("atom:id", ns)
            summary_el= entry.find("atom:summary", ns)

            title = title_el.text.strip() if title_el is not None and title_el.text else "No title"
            abs_txt = summary_el.text.strip() if summary_el is not None and summary_el.text else ""
            raw_id = id_el.text.strip() if id_el is not None and id_el.text else ""
            # make pdf link from abs link
            if raw_id:
                pdf_link = raw_id.replace("/abs/", "/pdf/")
            else:
                pdf_link = ""

            results.append({"title": title, "link": pdf_link, "abstract": abs_txt})

        all_results[topic] = results if results else [{"title": "No results found", "link": "", "abstract": ""}]

    return all_results

In [23]:
def summarize_abstract_with_mistral_or_fallback(text, max_len=400):
    text = (text or "").strip()
    if not text:
        return " No abstract text available."

    prompt = f"Read the following abstract and provide a concise summary (not more than 3 bullet points). Avoid copying sentences directly:\n\n{text}"

    # if generate_text (Mistral) exists in globals, use it
    if "generate_text" in globals():
        try:
            # generate_text returns a list of strings per your earlier code
            out = generate_text(prompt, max_length=max_len)
            if isinstance(out, list) and out:
                return out[0].strip()
            elif isinstance(out, str):
                return out.strip()
        except Exception as e:
            # fall through to HF fallback
            fallback_error = f"(Mistral generate failed: {e})\n"

    # Fallback: use HuggingFace summarizer (light handling)
    try:
        from transformers import pipeline
        summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
        # keep within model input limits — chunk if needed
        chunk = text[:3000]
        s = summarizer(chunk, max_length=80, min_length=50, do_sample=False)
        return s[0]["summary_text"].strip()
    except Exception as e:
        return f" Summarization failed: {e}"


In [24]:
#  3) Main flow
def run_search_and_summarize(topics, max_results_per_topic=3):
    papers_by_topic = search_arxiv_by_topic(topics, max_results=max_results_per_topic)
    final = []  # list of dicts {topic, title, link, abstract, summary}

    for topic, papers in papers_by_topic.items():
        for paper in papers:
            title = paper.get("title","")
            link  = paper.get("link","")
            abstract = paper.get("abstract","")
            summary = summarize_abstract_with_mistral_or_fallback(abstract)
            final.append({
                "topic": topic,
                "title": title,
                "link": link,
                "abstract": abstract,
                "summary": summary
            })
    return final


In [25]:
# 4) Run 
topics = globals().get("topics", ["deep learning", "epilepsy"])  # if you already have `topics` from Mistral it will be used
results = run_search_and_summarize(topics, max_results_per_topic=2)

# ----- 5) Print nicely -----
for r in results:
    print("\n📌 Topic:", r["topic"])
    print("🔗 Title:", r["title"])
    print("🔗 PDF link:", r["link"])
    print("📝 Abstract (first 300 chars):", (r["abstract"] or "")[:300].replace("\n", " "), "...")
    print("✅ Summary:\n", r["summary"])
    print("="*60)



📌 Topic: Symmetricity-Driven Deep Learning Framework
🔗 Title: Moving Deep Learning into Web Browser: How Far Can We Go?
🔗 PDF link: http://arxiv.org/pdf/1901.09388v2
📝 Abstract (first 300 chars): Recently, several JavaScript-based deep learning frameworks have emerged, making it possible to perform deep learning tasks directly in browsers. However, little is known on what and how well we can do with these frameworks for deep learning in browsers. To bridge the knowledge gap, in this paper, w ...
✅ Summary:
 Read the following abstract and provide a concise summary (not more than 3 bullet points). Avoid copying sentences directly:

Recently, several JavaScript-based deep learning frameworks have emerged,
making it possible to perform deep learning tasks directly in browsers.
However, little is known on what and how well we can do with these frameworks
for deep learning in browsers. To bridge the knowledge gap, in this paper, we
conduct the first empirical study of deep learning in brow

In [ ]:
NGROK_TOKEN = ""
API_KEY = "Test11"

In [27]:
pip install pyngrok

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [28]:
from fastapi import FastAPI, Request, HTTPException
import uvicorn, threading, time, socket
from pyngrok import ngrok, conf

app = FastAPI()

@app.post("/generate")
async def gen(req: Request):
    if req.headers.get("authorization") != f"Bearer {API_KEY}":
        raise HTTPException(status_code=401, detail="Unauthorized")
    data = await req.json()
    return {
        "response": generate_text(
            data.get("prompt", ""),
            data.get("max_length", 300)
        )
    }

In [29]:
def free_port():
    s = socket.socket()
    s.bind(('', 0))
    port = s.getsockname()[1]
    s.close()
    return port

port = free_port()
conf.get_default().auth_token = NGROK_TOKEN
public_url = ngrok.connect(port).public_url
print("Your public URL:", public_url)

def run(): uvicorn.run(app, host="0.0.0.0", port=port)
threading.Thread(target=run, daemon=True).start()
time.sleep(1)

Your public URL: https://60288d3b0b6b.ngrok-free.app                                                


INFO:     Started server process [36]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:38625 (Press CTRL+C to quit)


INFO:     156.217.239.111:0 - "POST /generate HTTP/1.1" 200 OK
